In [51]:
from pyspark.sql import SparkSession
from datetime import datetime, timezone

ts = datetime.now(timezone.utc).strftime("%Y%m%d_%H%M%S")

spark = SparkSession.builder \
    .appName(f"Silver-Purchase-{ts}") \
    .getOrCreate()

In [52]:
from pyspark.sql.functions import current_timestamp, to_utc_timestamp, col
from pyspark.sql.types import DecimalType

In [53]:
#first execution 

df_bronze = spark.read.parquet("data_lake/bronze/purchase")
df_bronze.createOrReplaceTempView("purchase_bronze")

df_purchase_silver = spark.sql("""
WITH 
base AS (
    SELECT 
        a.transaction_datetime,
        a.transaction_date,
        a.purchase_id,
        a.buyer_id,
        a.prod_item_id,
        a.order_date,
        a.release_date,
        a.producer_id,
        CASE 
            WHEN a.release_date IS NOT NULL THEN 'Invoiced'
            ELSE 'Not Invoiced'
        END AS invoiced_status,
        ROW_NUMBER() OVER (
            PARTITION BY a.purchase_id, a.transaction_datetime
            ORDER BY a.transaction_datetime DESC
        ) AS rn,
        to_utc_timestamp(current_timestamp(), 'UTC') AS line_created_at,
        a.ingestion_date AS bronze_ingestion_date
    FROM purchase_bronze a
)
SELECT *
FROM base
WHERE rn = 1
""")

# Salvar como Delta Table
df_purchase_silver.coalesce(1).write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy("transaction_date") \
    .save("data_lake/silver/purchase")

In [54]:
from pyspark.sql.functions import (
    col,
    max as max_,
    row_number,
    when,
    current_timestamp,
    to_utc_timestamp
)
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException

SILVER_PATH = "data_lake/silver/purchase"
BRONZE_PATH = "data_lake/bronze/purchase"

# =====================================================
# 1. Ler Silver atual (se existir)
# =====================================================
try:
    df_silver_current = spark.read.parquet(SILVER_PATH)
except AnalysisException:
    df_silver_current = None


# =====================================================
# 2. Última ingestion processada
# =====================================================
last_ingestion = (
    df_silver_current
    .agg(max_("bronze_ingestion_date").alias("max_date"))
    .first()["max_date"]
) if df_silver_current is not None else None


# =====================================================
# 3. Ler Bronze (incremental ou full)
# =====================================================
df_bronze = spark.read.parquet(BRONZE_PATH)

df_bronze_incremental = (
    df_bronze.filter(col("ingestion_date") > last_ingestion)
    if last_ingestion
    else df_bronze
)


# =====================================================
# 4. Transformações Bronze → Silver
# =====================================================
df_bronze_ready = (
    df_bronze_incremental
    .withColumn(
        "invoiced_status",
        when(col("release_date").isNotNull(), "Invoiced")
        .otherwise("Not Invoiced")
    )
    .withColumn(
        "line_created_at",
        to_utc_timestamp(current_timestamp(), "UTC")
    )
    .withColumnRenamed(
        "ingestion_date",
        "bronze_ingestion_date"
    )
)


# =====================================================
# 5. Union Silver + Bronze incremental
# =====================================================
df_union = (
    df_silver_current.drop("rn").unionByName(df_bronze_ready)
    if df_silver_current is not None
    else df_bronze_ready
)


# =====================================================
# 6. Deduplicação por evento (CDC)
# =====================================================
event_window = Window.partitionBy(
    "purchase_id",
    "transaction_datetime"
).orderBy(
    col("bronze_ingestion_date").desc()
)

df_silver_dedup = (
    df_union
    .withColumn("rn", row_number().over(event_window))
    .filter(col("rn") == 1)
    .drop("rn")
)


# =====================================================
# 7. Último estado por purchase
# =====================================================
purchase_window = Window.partitionBy(
    "purchase_id"
).orderBy(
    col("transaction_datetime").desc()
)

df_silver_final = (
    df_silver_dedup
    .withColumn("rk", row_number().over(purchase_window))
    .withColumn(
        "is_latest",
        col("rk") == 1
    )
    .drop("rk")
)


In [55]:
# =====================================================
# 7. Escrita final da Silver
# =====================================================
df_silver_final.write \
    .format("parquet") \
    .mode("overwrite") \
    .partitionBy("transaction_date") \
    .save("data_lake/silver/purchase")

In [56]:
# Criar view temporária
df_silver_final = spark.read.parquet("data_lake/silver/purchase")
df_silver_final.createOrReplaceTempView("purchase_silver")

spark.sql("SELECT * FROM purchase_silver").show()

+--------------------+-----------+--------+------------+----------+------------+-----------+---------------+--------------------+---------------------+---------+----------------+
|transaction_datetime|purchase_id|buyer_id|prod_item_id|order_date|release_date|producer_id|invoiced_status|     line_created_at|bronze_ingestion_date|is_latest|transaction_date|
+--------------------+-----------+--------+------------+----------+------------+-----------+---------------+--------------------+---------------------+---------+----------------+
| 2023-07-15 09:00:00|         55|  160001|           5|2023-01-20|  2023-03-01|     852852|       Invoiced|2026-01-14 22:33:...| 2026-01-14 16:21:...|     true|      2023-07-15|
| 2023-02-26 03:00:00|         69|  160001|          18|2023-01-26|  2023-02-28|      96967|       Invoiced|2026-01-14 22:33:...| 2026-01-14 16:21:...|     true|      2023-02-26|
| 2023-02-05 10:00:00|         55|  160001|           5|2023-01-20|  2023-01-20|     852852|       Invoic